# 📦 IPFS + Blockchain Stress Test

Upload synthetic payloads repeatedly to IPFS, then commit each CID to the chain. Useful for testing:
- IPFS throughput / capacity
- Chain stub / real contract call frequency
- Error handling when IPFS or chain is slow/unavailable


In [ ]:
# Imports
import time
import numpy as np
from polyscale_fl.ipfs.ipfs_client import IPFSClient
from polyscale_fl.chain.chain_stub import ChainStub
import json


In [ ]:
# Setup clients
ipfs = IPFSClient()
chain = ChainStub()


## Generate synthetic payloads
Create random matrices or dummy model weights for upload.

In [ ]:
def make_dummy_state(dim=100):
    return {f"param_{i}": np.random.randn(dim).astype(np.float32) for i in range(5)}


## Run stress loop
Specify number of iterations and delay between them.

In [ ]:
iterations = 20
delay_sec = 0.5

results = []
for i in range(iterations):
    state = make_dummy_state(200)
    # Serialize and upload to IPFS
    try:
        cid = ipfs.add_state(state)
        ts = time.time()
        chain.register_model(cid, i, {"note": "stress_test"})
        results.append({"iteration": i, "cid": cid, "time": ts, "status": "ok"})
        print(f"[{i}] Uploaded CID {cid}")
    except Exception as e:
        results.append({"iteration": i, "error": str(e)})
        print(f"[{i}] Error: {e}")
    time.sleep(delay_sec)

# Save results
with open("stress_results.json", "w") as f:
    json.dump(results, f, indent=2)
print("Stress test finished, results saved to stress_results.json")